<a href="https://colab.research.google.com/github/shivanishimpi/MachineLearning_PyTorch/blob/master/PyTorch_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#@title Imports
import numpy as np
import torch

```PyTorch``` is for processing tensors (Numbers, Vectors, 3 Dimensional arrays)

Reference Links:


[For more information on Tensors](https://pytorch.org/docs/stable/tensors.html)

[PyTorch AutoGrad](https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95)

#Table of Contents

[Basics of PyTorch](#scrollTo=0eg8vAOQzChK)

[Linear Regression from Scratch](#scrollTo=FOtz5Oy3F0H9)

[Linear Regression using PyTorch built-ins](#scrollTo=eFPRLkl_-V0J)

#Basics of PyTorch

Working with Torch tensors 


In [8]:
#number
tensor1 = torch.tensor(4.)
tensor1.dtype
#vector
tensor1 = torch.tensor([1,3.5,2,4,3.])
tensor1.dtype
#matrix
tensor1 = torch.tensor([[1,3],[2,4.],[2,4]])
tensor1
#3darray
tensor1 = torch.tensor(
    [
     [[1,2],[4,5]],
     [[2,3],[23,34]]
    ]
)
tensor1.shape

torch.Size([2, 2, 2])

Operations and gradients in Tensors

In [122]:
'''
m = torch.tensor([2,3,4])
x = torch.tensor([42,35,62])
c = torch.tensor([4,5,6.], requires_grad=True) #only floats need gradients
y=m*x+c 
print(f'Operation: {y}')
'''

"\nm = torch.tensor([2,3,4])\nx = torch.tensor([42,35,62])\nc = torch.tensor([4,5,6.], requires_grad=True) #only floats need gradients\ny=m*x+c \nprint(f'Operation: {y}')\n"

In [10]:
m = torch.tensor(6. ,requires_grad=True)
x = torch.tensor(2., requires_grad=True)
c = torch.tensor(4., requires_grad=True) #only floats need gradients
y=m*x+c
y.backward() #works only for scalars, computes gradients

print(f'''Displaying gradients / derivatives:
y = mx + c = {y}
dy/dx = {x.grad}
dy/dm = {m.grad}
dy/dc = {c.grad}
''')

Displaying gradients / derivatives:
y = mx + c = 16.0
dy/dx = 6.0
dy/dm = 2.0
dy/dc = 1.0



Interoperability with ```Numpy```

In [11]:
import numpy as np

print(f'Input Torch Tensor: {y.dtype}')

#from torch tensor to numpy array
x = y.detach().numpy() #for gradients, else y.numpy()
print(f'Converted to Numpy Array: {x.dtype}')

#from numpy array to torch tensor
y = torch.from_numpy(x)
print(f'Reconverted to Torch Tensor: {y.dtype}')

Input Torch Tensor: torch.float32
Converted to Numpy Array: float32
Reconverted to Torch Tensor: torch.float32


#Linear Regression from the Scratch

We aren't using any input files here, since this is just s test implementation of model so we are going to create our own data

*Data creation*

In [92]:
#Not gonna work since np.random.uniform doesn't take the dtype argument and it'd naturally be in float64
'''
inputs = np.random.uniform(low=20., high=99., size=(5,3))
labels = np.random.uniform(low=50., high=150., size=(5,2))
print(f'Inputs: {inputs}\nLabels: {labels}')
'''

"\ninputs = np.random.uniform(low=20., high=99., size=(5,3))\nlabels = np.random.uniform(low=50., high=150., size=(5,2))\nprint(f'Inputs: {inputs}\nLabels: {labels}')\n"

In [93]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
labels = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [94]:
inputs = torch.from_numpy(inputs)
labels = torch.from_numpy(labels)

In [95]:
inputs.dtype #if dtype!='float32' print(runtimeError)

torch.float32

Creating the linear regresssion model 


The equations are going to be - 



$$labels^{1} = w_{11}\cdot inputs^{1} + w_{12}\cdot inputs^{2}+ w_{13}\cdot inputs^{3}+b_{1}$$

$$labels^{2} = w_{21}\cdot inputs^{1} + w_{22}\cdot inputs^{2}+ w_{23}\cdot inputs^{3}+b_{2}$$

In [96]:
weights = torch.randn(2,3,requires_grad=True) #2x3 weights matrix
bias = torch.randn(2,requires_grad=True) #two bias units
print(f'''
{weights}
{bias}
''')


tensor([[ 0.3221,  1.9652,  1.0937],
        [ 0.6828, -0.0392,  0.7854]], requires_grad=True)
tensor([ 0.7780, -0.3254], requires_grad=True)



In [97]:
def model(x):
  return x@weights.t() + bias #@ is matrix multiplication and .t() is transpose

In [106]:
predictions = model(inputs)
print(predictions)
print(labels)

tensor([[202.9915,  80.6637],
        [273.0264, 108.6238],
        [355.5762,  99.3747],
        [158.6064,  96.6955],
        [288.2230,  97.9998]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Mean Squared Error (MSE) Loss

In [111]:
#delta = (predictions - labels)**2
#print(torch.sum(delta)/delta.numel())

tensor(17272.4844, grad_fn=<DivBackward0>)


In [112]:
def MSE(val1,val2):
  delta = (val1-val2)**2
  return torch.sum(delta)/delta.numel()

In [114]:
loss = MSE(predictions, labels)
print(loss)

tensor(17272.4844, grad_fn=<DivBackward0>)


In [118]:
loss.backward(create_graph=True)

PyTorch accumulates gradients, so if ```.backward()``` is called again, it'll add the new gradients to the former one, rather than replacing it

In [133]:
weights.grad.zero_()
bias.grad.zero_()

tensor([0., 0.], grad_fn=<ZeroBackward>)

Gradient descent

```torch.no_grad()``` avoids any gradient modification while updating the weights and biases

In [139]:
numEpoch = 100
learning_rate = 1e-4

for i in range(numEpoch):
  predictions = model(inputs)
  loss = MSE(predictions,labels)
  loss.backward()
  with torch.no_grad():
    weights -= weights.grad * learning_rate
    bias -= bias.grad * learning_rate
    weights.grad.zero_()
    bias.grad.zero_()

In [140]:
predictions = model(inputs)
loss = MSE(predictions, labels)
print(loss)

tensor(0.5599, grad_fn=<DivBackward0>)


In [141]:
print(predictions)
print(labels)

tensor([[ 57.3169,  70.2572],
        [ 81.9331, 100.6729],
        [119.0743, 133.0346],
        [ 21.1779,  37.0412],
        [101.5394, 119.0618]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


#Linear Regression Using PyTorch built-ins

https://youtu.be/vo_fUOk-IKk?list=PLWKjhJtqVAbm3T2Eq1_KgloC7ogdXxdRa&t=5567